# Implementing and fitting a simple syrinx model

Model based on [Mindlin et al., 2003](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.68.041908). 

We want to fit the system of first-order nonlinear equations:

$$
\dot{x} = y \\
\dot{y} = -\epsilon x - C x^2 y + By - D_0
$$
where
$$
\epsilon = \epsilon_1 + \epsilon_2 K(t) \\
B = \beta_1 + \beta_2 P(t) \\
D_0 = \delta D(t)
$$
and $K(t)$, $D(t)$, and $P(t)$ are the (linear envelopes of) tension in the ventral syringeal muscle (vS), the tracheobronchialis dorsalis (dTB), and sub-syringeal air pressure, respectively.

From the original paper, we take parameter values
$$
\epsilon_1 = 1.25 \times 10^8 \, \mathrm{s}^{-2} \\
\epsilon_2 = 7.5 \times 10^9 \, \mathrm{V}^{-1}\cdot \mathrm{s}^{-2} \\
C = 2 \times 10^8 \, \mathrm{cm}^{-2} \cdot \mathrm{s}^{-1} \\
\beta_1 = -2 \times 10^3 \, \mathrm{s}^{-1} \\
\beta_2 = 5.3 \times 10^4 \, \mathrm{V}^{-1}\cdot \mathrm{s}^{-1} \\
\delta = 15 \times 10^6 \, \mathrm{cm}\cdot\mathrm{V}^{-1} \cdot \mathrm{s}^{-2}
$$

In [ ]:
eps1 = 1.25e8
eps2 = 7.5e9
C = 2e8
beta1 = -2e3
beta2 = 5.3e4
delta = 15e6

But whereas the original paper used measured values for $K$, $P$, and $D$, we use simpler time series constructed to have the same shape:

In [ ]:
import jax
import jax.numpy as jnp
from jax import lax
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

In [ ]:
sr = 4e4  # data sampling rate (Hz)
T = 2
t_axis = jnp.arange(0, T, 1/sr)

In [ ]:
def make_pulse_sequence(pulse_fun, arg_list):
    def applicator(carry, args):
       t = carry
       return t, pulse_fun(t, *args)
    
    def scan_and_sum(t):
        _, out = lax.scan(applicator, t, arg_list)
        return jnp.sum(out)

    return scan_and_sum

In [ ]:
def make_pressure_pulse(freq, amp, cutoff, dc):
    def bump(t, loc):
        return lax.cond(
            jnp.abs(t - loc) > 0.5/freq,
            lambda t: amp * cutoff + dc,
            lambda t: amp * jnp.maximum(jnp.cos(2 * jnp.pi * freq * (t - loc)), cutoff) + dc,
            t 
        )

    return bump

pfreq = 2  # pressure pulse frequency (Hz)
pA = 0.015  # pressure amplitude (Volts)
p0 = 0.005  # pressure DC offset (Volts)
pcutoff = -.7
plocs = jnp.array([0.3, 0.9])
bump = make_pressure_pulse(pfreq, pA, pcutoff, p0)
P = jax.vmap(make_pulse_sequence(bump, (plocs,))) #, in_axes=(0, None))

plt.plot(t_axis, P(t_axis))

In [ ]:
def make_tension_pulse_fn(shape=1, scale=1, peak=1):
    norm = peak * jnp.exp(shape - shape * jnp.log(shape) - shape * jnp.log(scale)) 

    fn = lambda t, loc: norm * jnp.exp((t - loc)/scale) * jnp.maximum((loc - t), 0)**shape 

    return fn
    
        
kshape = 5  # shape parameter of gamma function
kscale = 0.025  # rate parameter of gamma function (s)
kpeak = 0.06  # peak value (Volts)
klocs = jnp.array([0, 0.5, 1, 1.5])

pulse = make_tension_pulse_fn(shape=kshape, scale=kscale, peak=kpeak)
K = jax.vmap(make_pulse_sequence(pulse, (klocs,)))

plt.plot(t_axis, K(t_axis))

In [ ]:
dfreq = 2 
dlocs1 = jnp.arange(0.1, t_axis[-1], 1/dfreq)  # pressure pulse frequency (Hz)
dlocs2 = jnp.arange(0.45, t_axis[-1], 1/dfreq)  # pressure pulse frequency (Hz)
dlocs = jnp.sort(jnp.concatenate([dlocs1, dlocs2]))
dA = jnp.array([0.05, 0.02, 0.01, 0.05, 0.03, 0.04, 0.05, 0.05])
dwid = 0.01

gpulse = lambda t, loc, amp: amp * jnp.exp(-0.5 * (t - loc)**2/dwid**2)

D = jax.vmap(make_pulse_sequence(gpulse, (dlocs, dA)))

plt.plot(t_axis, D(t_axis))

In [ ]:
plt.plot(t_axis, K(t_axis), label='vS tension')
plt.plot(t_axis, D(t_axis), label='dTB tension')
plt.plot(t_axis, P(t_axis), label='Pressure')
plt.xlabel("time (s)")
plt.ylabel("envelope (V)")
plt.xlim([0, 1.2])
plt.legend();

Now let's define and integrate the ODE:

In [ ]:
import diffrax

In [ ]:
def gradfun(t, y, args):
    K, D, P = args
    t_arr = jnp.array([t])
    eps = eps1 + eps2 * K(t_arr)
    B = beta1 + beta2 * P(t_arr) * 10
    D0 = delta * D(t_arr)

    xdot = y[1] 
    ydot = -eps * y[0] - C * y[0]**2 * y[1] + B * y[1] - D0

    return jnp.array((xdot, ydot[0]))


term = diffrax.ODETerm(gradfun)
solver = diffrax.Dopri5()
saveat = diffrax.SaveAt(ts=jnp.linspace(0, 1.2, int(sr)))
stepsize_controller = diffrax.PIDController(rtol=1e-5, atol=1e-5)

In [ ]:
soln = diffrax.diffeqsolve(term, solver, t0=0, t1=1.2, dt0=0.5/sr, y0=jnp.array((0, 0)), saveat=saveat,
                  stepsize_controller=stepsize_controller, args=(K, D, P), max_steps=100000)

In [ ]:
plt.plot(soln.ts, soln.ys[:, 0])

In [ ]:
audio = jnp.interp(t_axis, soln.ts, soln.ys[:, 0])

In [ ]:
from scipy.signal import stft

freqs, times, spec = stft(audio, fs=sr, 
                  nperseg=512, 
                  noverlap=480)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].plot(t_axis, K(t_axis), label='vS tension')
axs[0].plot(t_axis, D(t_axis), label='dTB tension')
axs[0].plot(t_axis, P(t_axis), label='Pressure')
axs[0].set_xlabel("time (s)")
axs[0].set_ylabel("envelope (V)")
axs[0].set_xlim([0, 1.2])
axs[0].legend();

axs[1].imshow(jnp.abs(spec), extent=[times[0], times[-1], freqs[0], freqs[-1]], aspect='auto')
axs[1].set_xlabel('time (s)')
axs[1].set_ylabel('frequency (Hz)')
axs[1].set_ylim([12500, 19000])
axs[1].set_xlim([0, 1.2])

plt.tight_layout()